## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.insert(0,'/Users/David/Git/World_Weather_Analysis') # Comment out this code if config is at same level
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Chelno-Vershiny,RU,54.4333,51.1000,24.91,91,100,11.12,light snow
1,Husavik,IS,66.0449,-17.3389,37.31,84,14,12.71,few clouds
2,Hualmay,PE,-11.0964,-77.6139,74.30,73,100,5.88,overcast clouds
3,Tasiilaq,GL,65.6145,-37.6368,28.56,96,100,13.80,snow
4,Punta Arenas,CL,-53.1500,-70.9167,50.11,66,75,33.38,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hualmay,PE,74.30,overcast clouds,-11.0964,-77.6139,
6,Hobart,AU,70.05,clear sky,-42.8794,147.3294,
8,Goure,NE,77.23,overcast clouds,13.9835,10.2704,
10,Butaritari,KI,81.61,broken clouds,3.0707,172.7902,
13,Lorengau,PG,81.00,heavy intensity rain,-2.0226,147.2712,
15,Bengkulu,ID,75.97,overcast clouds,-3.8004,102.2655,
18,Kapaa,US,73.72,light rain,22.0752,-159.3190,
19,Raga,SS,79.09,few clouds,8.4596,25.6780,
20,Male,MV,80.55,broken clouds,4.1748,73.5089,
23,Namibe,AO,73.76,broken clouds,-15.1961,12.1522,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f"Success and current index {index}")
    except IndexError:
        print("Hotel Not found")
        continue
        

Success and current index 2
Success and current index 6
Hotel Not found
Success and current index 10
Success and current index 13
Success and current index 15
Success and current index 18
Hotel Not found
Success and current index 20
Success and current index 23
Success and current index 30
Success and current index 36
Success and current index 37
Success and current index 40
Success and current index 43
Success and current index 44
Success and current index 46
Success and current index 47
Success and current index 49
Success and current index 51
Success and current index 55
Success and current index 56
Success and current index 59
Success and current index 60
Success and current index 62
Success and current index 66
Success and current index 70
Success and current index 74
Success and current index 75
Success and current index 78
Success and current index 79
Success and current index 82
Success and current index 85
Success and current index 92
Hotel Not found
Success and current index 

In [18]:
# 7. Drop the rows where there is no Hotel Name.
# Blank hotel names are not NA but "", use a boolean != to remove ""
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hualmay,PE,74.30,overcast clouds,-11.0964,-77.6139,Hotel Villa Kitzia Huacho
6,Hobart,AU,70.05,clear sky,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
10,Butaritari,KI,81.61,broken clouds,3.0707,172.7902,Isles Sunset Lodge
13,Lorengau,PG,81.00,heavy intensity rain,-2.0226,147.2712,Lorengau Harbourside Hotel
15,Bengkulu,ID,75.97,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
18,Kapaa,US,73.72,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Male,MV,80.55,broken clouds,4.1748,73.5089,Hotel Octave Maldives
23,Namibe,AO,73.76,broken clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
30,Georgetown,MY,78.73,few clouds,5.4112,100.3354,Cititel Penang
36,Manadhoo,MV,80.89,few clouds,5.7667,73.3833,Bon Abri maldives


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))